In [99]:
#@@@@@@@@@@@@@@@@@@@
#Bayesian regression
#Jeremy Kedziora
#4/20/2016
#@@@@@@@@@@@@@@@@@@@

import numpy as np    #import numpy
import scipy    #import scipy
from scipy.stats import invgamma as IG    #import scipy stats for the inverse gamma
import matplotlib.pyplot as plt    #import to visualize and plot
from math import ceil    #import the ceiling function




In [100]:
def beta_sampler(X,y,beta,sigma2):
    
    """A function to sample a single draw of the betas from the posterior in a bayesian regression.
    Take as inputs:
    X: a numpy array of independent variables
    y: a numpy array of the dependent variable
    beta: a list of lists of initial values for beta
    sigma2: a list of the initial value for sigma2
    mu: an array of prior means for the parameters
    t2: an array of prior variances for the parameters
    """
    t = len(beta[0])    #the time step
    mean = np.linalg.inv(X.dot(X.T)).dot(X.dot(y))    #compute the mean of the multivariate normal
    Sigma_matrix = sigma2[t-1]*np.linalg.inv(X.dot(X.T))    #compute the sigma matrix
    draw = np.random.multivariate_normal(mean,Sigma_matrix)    #draw from the posterior of beta
    
    for j in range(len(beta)):    #loop over beta parameters
        beta[j] = beta[j] + [draw[j]]    #and add the parameter to the list
        
    return beta

In [101]:
def sigma2_sampler(X,y,beta,sigma2):
    
    """A function to sample a single draw of the sigma2 from the posterior in a bayesian regression.
    Take as inputs:
    X: a numpy array of independent variables
    y: a numpy array of the dependent variable
    beta: a list of lists of initial values for beta
    sigma2: a list of the initial value for sigma2
    mu: an array of prior means for the parameters
    t2: an array of prior variances for the parameters
    """
    
    t = len(beta[0])    #the time step
    b = []    #initialize a list to hold the current values of b
    for beta_j in beta:    #loop over the variable effects
        b.append(beta_j[t-1])    #and take the last draw for each variable
    b = np.array(b)    #make them into an array for matrix multiplication
    parameter_1 = (X.shape[1] - X.shape[0])/2.0    #the first parameter in the inverse gamma
    parameter_2 = sum((y - X.T.dot(b))**2.0)/2.0    #the second parameter in the inverse gamma
    draw = 1.0/np.random.gamma(parameter_1,1.0/parameter_2)    #draw from the posterior distribution
    
    return sigma2 + [draw]

In [102]:
def MCMC(n_sim,X,y):
    
    """A function to run Gibb's sampling to estimate the posterior of linear model parameters.
    Take as inputs:
    n_sim: how many simulations you want to run
    X: a numpy array of independent variables
    y: a numpy array of the dependent variable
    beta: a list of lists of initial values for beta
    sigma2: a list of the initial value for sigma2
    mu: an array of prior means for the parameters
    t2: an array of prior variances for the parameters    
    """
    
    #specify initial values for beta and sigma
    beta_t = [[0.0]]*X.shape[0]    #set betas initially to zero
    sigma2_t = [1.0]    #set sigma2 initially to 1.0
    for _ in range(n_sim):    #loop over simulations
        beta_t = beta_sampler(X,y,beta_t,sigma2_t)    #sample beta
        sigma2_t = sigma2_sampler(X,y,beta_t,sigma2_t)    #sample sigma2
    
    return [beta_t,sigma2_t]

In [108]:
def posterior_summary(posterior,path):
    """A function to compute posterior summary statistics.
    Take as inputs:
    beta: a numpy array of samples from the posterior distribution
    """
    
    beta_means = np.mean(posterior,1)    #compute the mean of the posterior
    beta_HPD = [np.percentile(posterior,0.975,1),np.percentile(posterior,0.025,1)]    #compute the 95% region of highest posterior density
    p_positive = np.sum(np.array(posterior)>0,1)/len(posterior[0])    #compute the probability the parameter is positive
    for j in range(len(beta_means)):    #loop over variables
        print('For','x'+str(j),'the posterior mean is:',round(beta_means[j],4),'.')    #print the mean of the posterior
        print('The 95% HPD is: [',round(beta_HPD[1][j],4),round(beta_HPD[0][j],4),'].')    #print the 95% CI
        print('The probability that the effect is positive is: ',round(p_positive[j],4),'.')    #print the probability that the parameter has a positive effect
        print('____________________________________________________________________')
    
    plt.figure(figsize = (20,10))    #initiate the plot
    n_plots = ceil(len(beta_means)**0.5)    #how many plots to make
    for j in range(len(posterior)):    #loop over variables
        plt.subplot(n_plots,n_plots,j+1)    #plot subplot 1,1 in the 3x3 area
        plt.plot(posterior[j],'b-',lw=4)    #generate the plot
        plt.xlabel('Gibbs Sampler Iteration',fontsize=20)    #add an x label
        plt.ylabel(r'$\beta$'+str(j),fontsize=20)    #add a y label

    plt.savefig(path)    #save the plot to file    

In [110]:
#make fake data
N = 1000    #the number of observations
x_1 = np.random.normal(0,1,N)    #the independent variable
x_2 = np.random.normal(0,1,N)    #the independent variable
x_3 = np.random.normal(0,1,N)    #the independent variable
x_4 = np.random.normal(0,1,N)    #the independent variable
e = np.random.normal(0,1,N)    #the error
y = 1.0 + 2.0*x_1 - 1.5*x_2 - 1.0*x_3 + 1.75*x_4 + e    #the dependent variable
X = np.array([[1.0]*N,x_1,x_2,x_3,x_4])    #put the independent variables together

#call the MCMC routine
n_sim = 1000    #specify the number of simulations
Posterior = MCMC(n_sim,X,y)    #sample from the posterior

#call the summary function to make inferences
path = '/Users/seniordatascientist/Desktop/Metis Codes/Posterior.pdf'    #specify where I want to write graphs
posterior_summary(Posterior[0],path)    #compute summary stats

For x0 the posterior mean is: 1.0028 .
The 95% HPD is: [ 0.2263 0.9296 ].
The probability that the effect is positive is:  0.999 .
____________________________________________________________________
For x1 the posterior mean is: 2.0127 .
The 95% HPD is: [ 0.4724 1.9301 ].
The probability that the effect is positive is:  0.999 .
____________________________________________________________________
For x2 the posterior mean is: -1.5025 .
The 95% HPD is: [ -1.6173 -1.58 ].
The probability that the effect is positive is:  0.0 .
____________________________________________________________________
For x3 the posterior mean is: -0.9905 .
The 95% HPD is: [ -1.0888 -1.0719 ].
The probability that the effect is positive is:  0.0 .
____________________________________________________________________
For x4 the posterior mean is: 1.7767 .
The 95% HPD is: [ 0.419 1.7021 ].
The probability that the effect is positive is:  0.999 .
____________________________________________________________________
